In [42]:
import tensorflow

from sklearn.preprocessing import normalize
import pandas as pd
import numpy as np
import tensorflow as tf
import random
df=pd.read_csv('C:/Users/yy2895/Desktop/RawData.csv')
d = df.values
ntotal=len(df)


d = normalize(d, axis=0, norm='l2')


resultu = []
np.random.rand(4)
# Return 100 results (for instance)
for i in range(ntotal):
    
    res = random.random()
    if res < 0.1:
        resultu.append(1)
    elif res < 0.2 and res>=0.1:
        resultu.append(2)
    elif res < 0.3 and res>=0.2:
        resultu.append(3)
    elif res < 0.4 and res>=0.3:
        resultu.append(4)
    elif res < 0.5 and res>=0.4:
        resultu.append(5)
    elif res < 0.6 and res>=0.5:
        resultu.append(6)
    elif res < 0.7 and res>=0.6:
        resultu.append(7)
    elif res < 0.8 and res>=0.7:
        resultu.append(8)
    else:
        resultu.append(9)
resultu=np.array(resultu)
trainset=[]
for i in range(1,8):
    toinsert=d[resultu==i].astype(np.float32)
    trainset.append(toinsert)
validationset=d[resultu==8].astype(np.float32)
testset=d[resultu==9].astype(np.float32)


#x = data

# Following Hinton-Salakhutdinov Architecture

# 3 hidden layers for encoder
n_encoder_h_1 = 14
n_encoder_h_2 = 9



#n_encoder_h_5 = 10


# 3 hidden layers for decoder
#n_decoder_h_1 = 10
n_decoder_h_1 = 14
n_decoder_h_2 = 19




# Parameters
learning_rate = 0.005

#batch_size = 7
display_step = 1

total_batch=7
training_epochs = 5000

In [43]:
#we store the variables here
log_files_path = 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt'
def layer_batch_normalization(x, n_out, phase_train):
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - n_out: integer, depth of input maps - number of sample in the batch 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - batch-normalized maps   
    
    
    """

    beta_init = tf.constant_initializer(value=0.0, dtype=tf.float32)
    beta = tf.get_variable("beta", [n_out], initializer=beta_init)
    
    gamma_init = tf.constant_initializer(value=1.0, dtype=tf.float32)
    gamma = tf.get_variable("gamma", [n_out], initializer=gamma_init)

    #tf.nn.moment: https://www.tensorflow.org/api_docs/python/tf/nn/moments
    #calculate mean and variance of x
    batch_mean, batch_var = tf.nn.moments(x, [0], name='moments')
    
    #tf.train.ExponentialMovingAverage:
    #https://www.tensorflow.org/api_docs/python/tf/train/ExponentialMovingAverage
    #Maintains moving averages of variables by employing an exponential decay.
    ema = tf.train.ExponentialMovingAverage(decay=0.9)
    ema_apply_op = ema.apply([batch_mean, batch_var])
    ema_mean, ema_var = ema.average(batch_mean), ema.average(batch_var)
    
    def mean_var_with_update():
        with tf.control_dependencies([ema_apply_op]):
            return tf.identity(batch_mean), tf.identity(batch_var)
        
    #tf.cond: https://www.tensorflow.org/api_docs/python/tf/cond
    #Return true_fn() if the predicate pred is true else false_fn()
    mean, var = tf.cond(phase_train, mean_var_with_update, lambda: (ema_mean, ema_var))
    
    
    #Here, we changesd the shape of x into [[[x1]],[[x2] ],....]

    reshaped_x = tf.reshape(x, [-1, 1, 1, n_out])
    normed = tf.nn.batch_norm_with_global_normalization(reshaped_x, mean, var, beta, gamma, 1e-3, True)
    
    return tf.reshape(normed, [-1, n_out])

def layer(x, weight_shape, bias_shape, phase_train):
    
    """
    Defines the network layers
    input:
        - x: input vector of the layer
        - weight_shape: shape the the weight maxtrix
        - bias_shape: shape of the bias vector
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector of the layer after the matrix multiplication and non linear transformation
    """
    
    #initialize weights
    weight_init = tf.random_normal_initializer(stddev=(1.0/weight_shape[0])**0.5)
    W = tf.get_variable("W", weight_shape, initializer=weight_init)
    
    bias_init = tf.constant_initializer(value=0)
    b = tf.get_variable("b", bias_shape, initializer=bias_init)

    logits = tf.matmul(x, W) + b
    
    #apply the non-linear function after the batch normalization
    return tf.nn.sigmoid(layer_batch_normalization(logits, weight_shape[1], phase_train))
def encoder(x, n_code, phase_train):
    
    """
    Defines the network encoder part
    input:
        - x: input vector of the encoder
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder)
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reduced dimension
    """
    
    with tf.variable_scope("encoder"):
        
        
        with tf.variable_scope("code"):
            output = layer(x, [19, n_code], [n_code], phase_train)


    return output

def decoder(x, n_code, phase_train):
    """
    Defines the network encoder part
    input:
        - x: input vector of the decoder - reduced dimension vector
        - n_code: number of neurons in the code layer (output of the encoder - input of the decoder) 
        - phase_train: boolean tf.Varialbe, true indicates training phase
    output:
        - output vector: reconstructed dimension of the initial vector
    """
    
    with tf.variable_scope("decoder"):
        
   
        with tf.variable_scope("output"):
            output = layer(x, [ n_code, 19], [19], phase_train)

    return output
def loss(output, x):
    """
    Compute the loss of the auto-encoder
    
    intput:
        - output: the output of the decoder
        - x: true value of the sample batch - this is the input of the encoder
        
        the two have the same shape (batch_size * num_of_classes)
    output:
        - loss: loss of the corresponding batch (scalar tensor)
    
    """
    
    with tf.variable_scope("training"):
        
        l2 = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x)), 1))
        train_loss = tf.reduce_mean(l2)
        train_summary_op = tf.summary.scalar("train_cost", train_loss)
        return train_loss, train_summary_op
def training(cost, global_step):
    """
    defines the necessary elements to train the network
    
    intput:
        - cost: the cost is the loss of the corresponding batch
        - global_step: number of batch seen so far, it is incremented by one 
        each time the .minimize() function is called
    """
    
    optimizer = tf.train.AdamOptimizer(learning_rate=0.001, beta1=0.9, beta2=0.999, epsilon=1e-08, use_locking=False, name='Adam')
    #optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.005)
    train_op = optimizer.minimize(cost, global_step=global_step)
    return train_op
def evaluate(output, x):
    """
    evaluates the accuracy on the validation set 
    input:
        -output: prediction vector of the network for the validation set
        -x: true value for the validation set
    output:
        - val_loss: loss of the autoencoder
        - in_image_op: input image 
        - out_image_op:reconstructed image 
        - val_summary_op: summary of the loss
    """
    
    with tf.variable_scope("validation"):
        
        #in_image_op = image_summary("input_image", x)
        
        #out_image_op = image_summary("output_image", output)
        
        l2_norm = tf.sqrt(tf.reduce_sum(tf.square(tf.subtract(output, x, name="val_diff")), 1))
        
        val_loss = tf.reduce_mean(l2_norm)
        
        val_summary_op = tf.summary.scalar("val_cost", val_loss)
        
        #return val_loss, in_image_op, out_image_op, val_summary_op
        return val_loss,  val_summary_op
"""
def image_summary(label, tensor):
    #tf.summary.image: https://www.tensorflow.org/api_docs/python/tf/summary/image
    #Outputs a Summary protocol buffer with images.

    tensor_reshaped = tf.reshape(tensor, [-1, 19, 1, 1])
    return tf.summary.image(label, tensor_reshaped)
"""
if __name__ == '__main__':
    print('we begin')

    #if a python file, please use the 4 lines bellow and comment the "n_code = '2'"
    #parser = argparse.ArgumentParser(description='Autoencoder')
    #parser.add_argument('n_code', nargs=1, type=str)
    #args = parser.parse_args(['--help'])
    #n_code = args.n_code[0]
    
    #if a jupyter file, please comment the 4 above and use the one bellow
    n_code = '14'
    
    #feel free to change with your own 
    #log_files_path = r'C:\Users\yy2895\Desktop\pys'
    
    with tf.Graph().as_default():

        with tf.variable_scope("autoencoder_model"):

            #the input variables are first define as placeholder 
            # a placeholder is a variable/data which will be assigned later 
            # image vector & label, phase_train is a boolean 
            x = tf.placeholder("float", [None, 19]) # MNIST data image of shape 28*28=784
            
            phase_train = tf.placeholder(tf.bool)
            
            #define the encoder 
            code = encoder(x, int(n_code), phase_train)
            
            #define the decoder
            output = decoder(code, int(n_code), phase_train)
            
            #compute the loss 
            cost, train_summary_op = loss(output, x)

            #initialize the value of the global_step variable 
            # recall: it is incremented by one each time the .minimise() is called
            global_step = tf.Variable(0, name='global_step', trainable=False)

            train_op = training(cost, global_step)

            #evaluate the accuracy of the network (done on a validation set)
            #eval_op, in_image_op, out_image_op, val_summary_op = evaluate(output, x)
            eval_op, val_summary_op = evaluate(output, x)

            summary_op = tf.summary.merge_all()

            #save and restore variables to and from checkpoints.
            #saver = tf.train.Saver(max_to_keep=200)
            saver = tf.train.Saver()

            #defines a session
            sess = tf.Session()

            # summary writer
            #https://www.tensorflow.org/api_docs/python/tf/summary/FileWriter
            #train_writer = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #val_writer   = tf.summary.FileWriter(log_files_path+'mnist_autoencoder_hidden_' + n_code + '_logs/', graph=sess.graph)

            #initialization of the variables
            init_op = tf.global_variables_initializer()

            sess.run(init_op)
            currentmin=10000000000
            currentmin_index=-1
            error_path=[]
            togive1=[]
            # Training cycle
            for epoch in range(training_epochs):

                avg_cost = 0.
                #total_batch = int(numberofsamples/batch_size)
                
                # Loop over all batches
                for i in range(total_batch):
                    
                    minibatch_x= trainset[i]
                    
                    # Fit training using batch data
                    #the training is done using the training dataset
                    _, new_cost, train_summary = sess.run([train_op, cost, train_summary_op], feed_dict={x: minibatch_x, phase_train: True})
                    
                    #train_writer.add_summary(train_summary, sess.run(global_step))
                    
                    # Compute average loss
                    avg_cost += new_cost/total_batch
                    
                
                # Display logs per epoch step
                if epoch % display_step == 0:
                    print("Epoch:", '%04d' % (epoch+1), "cost =", "{:0.9f}".format(avg_cost))

                    #the accuracy is evaluated using the validation dataset
                    #train_writer.add_summary(train_summary, sess.run(global_step))

                    #validation_loss, in_image, out_image, val_summary = sess.run([eval_op, in_image_op, out_image_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    validation_loss,  val_summary = sess.run([eval_op, val_summary_op], feed_dict={x: validationset, phase_train: False})
                    #val_writer.add_summary(in_image, sess.run(global_step))
                    #val_writer.add_summary(out_image, sess.run(global_step))
                    val_writer.add_summary(val_summary, sess.run(global_step))
                    print("Validation Loss:", validation_loss)
                    error_path.append(validation_loss)
                    
                    if validation_loss<currentmin:
                        currentmin=validation_loss
                        currentmin_index=epoch
                        saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                        togive7=[]
                        for i in range(len(d)):
                            any_image = d[i].reshape(-1,19)
                            output_any_image = sess.run(code,feed_dict={x:any_image,phase_train: False})
                            togive7.append(output_any_image)
                            
                        togive1=togive7
                    #save to use later
                    #https://www.tensorflow.org/api_docs/python/tf/train/Saver
                    #saver.save(sess, log_files_path+'model-checkpoint', global_step=global_step)
                    #saver.save(sess, 'C:/Users/yy2895/Desktop/st19-14-19/tmp/model.ckpt')
                    

            print("Optimization Finished!")

            test_loss = sess.run(eval_op, feed_dict={x: testset, phase_train: False})
            #nps_loss = sess.run(eval_op, feed_dict={x: mnist.validation.images, phase_train: False})
            print("Test Loss:", test_loss)
            

we begin
Epoch: 0001 cost = 2.267292806
Validation Loss: 2.7203662
Epoch: 0002 cost = 2.252367122
Validation Loss: 2.4011958
Epoch: 0003 cost = 2.237690415
Validation Loss: 2.3864565
Epoch: 0004 cost = 2.222809928
Validation Loss: 2.3749034
Epoch: 0005 cost = 2.206821578
Validation Loss: 2.3756406
Epoch: 0006 cost = 2.189512730
Validation Loss: 2.4179192
Epoch: 0007 cost = 2.170483317
Validation Loss: 2.4199295
Epoch: 0008 cost = 2.149913618
Validation Loss: 2.4094696
Epoch: 0009 cost = 2.128934281
Validation Loss: 2.3933074
Epoch: 0010 cost = 2.108378513
Validation Loss: 2.3756807
Epoch: 0011 cost = 2.088259833
Validation Loss: 2.3347232
Epoch: 0012 cost = 2.068148715
Validation Loss: 2.1176355
Epoch: 0013 cost = 2.047423261
Validation Loss: 2.0661752
Epoch: 0014 cost = 2.026686668
Validation Loss: 1.9976523
Epoch: 0015 cost = 2.008111630
Validation Loss: 1.9580992
Epoch: 0016 cost = 1.992551804
Validation Loss: 1.934895
Epoch: 0017 cost = 1.979146532
Validation Loss: 1.9245093
Epoch:

Validation Loss: 1.0956283
Epoch: 0143 cost = 1.051598838
Validation Loss: 1.1225846
Epoch: 0144 cost = 1.046685866
Validation Loss: 1.0921907
Epoch: 0145 cost = 1.041798592
Validation Loss: 1.0041512
Epoch: 0146 cost = 1.036936743
Validation Loss: 0.9792394
Epoch: 0147 cost = 1.032100303
Validation Loss: 1.017008
Epoch: 0148 cost = 1.027289118
Validation Loss: 1.0292032
Epoch: 0149 cost = 1.022502967
Validation Loss: 1.0588467
Epoch: 0150 cost = 1.017741680
Validation Loss: 1.0235628
Epoch: 0151 cost = 1.013005478
Validation Loss: 0.9723004
Epoch: 0152 cost = 1.008293637
Validation Loss: 0.94863105
Epoch: 0153 cost = 1.003606021
Validation Loss: 0.9651555
Epoch: 0154 cost = 0.998943065
Validation Loss: 0.95006496
Epoch: 0155 cost = 0.994304197
Validation Loss: 0.96968436
Epoch: 0156 cost = 0.989689572
Validation Loss: 0.9450514
Epoch: 0157 cost = 0.985098575
Validation Loss: 0.9717028
Epoch: 0158 cost = 0.980531607
Validation Loss: 0.9999402
Epoch: 0159 cost = 0.975988167
Validation L

Epoch: 0288 cost = 0.558806854
Validation Loss: 0.56392866
Epoch: 0289 cost = 0.556685226
Validation Loss: 0.55533737
Epoch: 0290 cost = 0.554576933
Validation Loss: 0.55889654
Epoch: 0291 cost = 0.552481949
Validation Loss: 0.5490822
Epoch: 0292 cost = 0.550400053
Validation Loss: 0.54847056
Epoch: 0293 cost = 0.548331397
Validation Loss: 0.55026275
Epoch: 0294 cost = 0.546275667
Validation Loss: 0.52801126
Epoch: 0295 cost = 0.544232862
Validation Loss: 0.542902
Epoch: 0296 cost = 0.542202950
Validation Loss: 0.5387671
Epoch: 0297 cost = 0.540185775
Validation Loss: 0.5390473
Epoch: 0298 cost = 0.538181279
Validation Loss: 0.53766036
Epoch: 0299 cost = 0.536189428
Validation Loss: 0.5384074
Epoch: 0300 cost = 0.534210069
Validation Loss: 0.53909206
Epoch: 0301 cost = 0.532243133
Validation Loss: 0.55161387
Epoch: 0302 cost = 0.530288458
Validation Loss: 0.540133
Epoch: 0303 cost = 0.528345977
Validation Loss: 0.5378961
Epoch: 0304 cost = 0.526415757
Validation Loss: 0.5337454
Epoch: 

Epoch: 0432 cost = 0.353918833
Validation Loss: 0.3660995
Epoch: 0433 cost = 0.353007951
Validation Loss: 0.35688862
Epoch: 0434 cost = 0.352102058
Validation Loss: 0.3534528
Epoch: 0435 cost = 0.351201249
Validation Loss: 0.34163556
Epoch: 0436 cost = 0.350305306
Validation Loss: 0.3344875
Epoch: 0437 cost = 0.349414404
Validation Loss: 0.3521777
Epoch: 0438 cost = 0.348528351
Validation Loss: 0.35241723
Epoch: 0439 cost = 0.347647203
Validation Loss: 0.35319898
Epoch: 0440 cost = 0.346770857
Validation Loss: 0.33797717
Epoch: 0441 cost = 0.345899343
Validation Loss: 0.32867715
Epoch: 0442 cost = 0.345032654
Validation Loss: 0.3400057
Epoch: 0443 cost = 0.344170707
Validation Loss: 0.33287618
Epoch: 0444 cost = 0.343313473
Validation Loss: 0.3428501
Epoch: 0445 cost = 0.342460960
Validation Loss: 0.33782902
Epoch: 0446 cost = 0.341613114
Validation Loss: 0.33777487
Epoch: 0447 cost = 0.340769849
Validation Loss: 0.32205483
Epoch: 0448 cost = 0.339931271
Validation Loss: 0.34723157
Epo

Epoch: 0572 cost = 0.264896544
Validation Loss: 0.27045295
Epoch: 0573 cost = 0.264484239
Validation Loss: 0.26950997
Epoch: 0574 cost = 0.264074521
Validation Loss: 0.25878075
Epoch: 0575 cost = 0.263667392
Validation Loss: 0.24898328
Epoch: 0576 cost = 0.263262798
Validation Loss: 0.27740887
Epoch: 0577 cost = 0.262860781
Validation Loss: 0.26201716
Epoch: 0578 cost = 0.262461247
Validation Loss: 0.25991854
Epoch: 0579 cost = 0.262064267
Validation Loss: 0.25793156
Epoch: 0580 cost = 0.261669802
Validation Loss: 0.25392112
Epoch: 0581 cost = 0.261277852
Validation Loss: 0.25437757
Epoch: 0582 cost = 0.260888389
Validation Loss: 0.25808856
Epoch: 0583 cost = 0.260501423
Validation Loss: 0.2554072
Epoch: 0584 cost = 0.260116926
Validation Loss: 0.24443929
Epoch: 0585 cost = 0.259734901
Validation Loss: 0.26553452
Epoch: 0586 cost = 0.259355343
Validation Loss: 0.26039565
Epoch: 0587 cost = 0.258978209
Validation Loss: 0.2645198
Epoch: 0588 cost = 0.258603552
Validation Loss: 0.26623473

...

Validation Loss: 0.03234966
Epoch: 4331 cost = 0.027205904
Validation Loss: 0.03360378
Epoch: 4332 cost = 0.027228909
Validation Loss: 0.031207401
Epoch: 4333 cost = 0.027205024
Validation Loss: 0.037203684
Epoch: 4334 cost = 0.027227991
Validation Loss: 0.037031077
Epoch: 4335 cost = 0.027204128
Validation Loss: 0.03655242
Epoch: 4336 cost = 0.027227103
Validation Loss: 0.053519573
Epoch: 4337 cost = 0.027203205
Validation Loss: 0.057322342
Epoch: 4338 cost = 0.027226229
Validation Loss: 0.053779203
Epoch: 4339 cost = 0.027202301
Validation Loss: 0.054639746
Epoch: 4340 cost = 0.027225370
Validation Loss: 0.036421534
Epoch: 4341 cost = 0.027201416
Validation Loss: 0.039539095
Epoch: 4342 cost = 0.027224453
Validation Loss: 0.055435997
Epoch: 4343 cost = 0.027200520
Validation Loss: 0.040998846
Epoch: 4344 cost = 0.027223555
Validation Loss: 0.03328844
Epoch: 4345 cost = 0.027199626
Validation Loss: 0.03437763
Epoch: 4346 cost = 0.027222659
Validation Loss: 0.03479902
Epoch: 4347 cost 

Epoch: 4475 cost = 0.027140766
Validation Loss: 0.06467884
Epoch: 4476 cost = 0.027164538
Validation Loss: 0.054838464
Epoch: 4477 cost = 0.027139887
Validation Loss: 0.038691737
Epoch: 4478 cost = 0.027163636
Validation Loss: 0.04701008
Epoch: 4479 cost = 0.027138977
Validation Loss: 0.03404746
Epoch: 4480 cost = 0.027162731
Validation Loss: 0.03618025
Epoch: 4481 cost = 0.027138057
Validation Loss: 0.039401658
Epoch: 4482 cost = 0.027161821
Validation Loss: 0.039170902
Epoch: 4483 cost = 0.027137157
Validation Loss: 0.048154756
Epoch: 4484 cost = 0.027160931
Validation Loss: 0.055598345
Epoch: 4485 cost = 0.027136242
Validation Loss: 0.050913546
Epoch: 4486 cost = 0.027160017
Validation Loss: 0.041967027
Epoch: 4487 cost = 0.027135347
Validation Loss: 0.041016236
Epoch: 4488 cost = 0.027159119
Validation Loss: 0.043008544
Epoch: 4489 cost = 0.027134421
Validation Loss: 0.03789891
Epoch: 4490 cost = 0.027158251
Validation Loss: 0.05261896
Epoch: 4491 cost = 0.027133515
Validation Loss

Validation Loss: 0.034784675
Epoch: 4616 cost = 0.027100652
Validation Loss: 0.03304956
Epoch: 4617 cost = 0.027075880
Validation Loss: 0.03507794
Epoch: 4618 cost = 0.027099742
Validation Loss: 0.03247294
Epoch: 4619 cost = 0.027074977
Validation Loss: 0.034838896
Epoch: 4620 cost = 0.027098786
Validation Loss: 0.03590319
Epoch: 4621 cost = 0.027074046
Validation Loss: 0.03290783
Epoch: 4622 cost = 0.027097894
Validation Loss: 0.031845026
Epoch: 4623 cost = 0.027073139
Validation Loss: 0.035486847
Epoch: 4624 cost = 0.027096985
Validation Loss: 0.040572148
Epoch: 4625 cost = 0.027072213
Validation Loss: 0.037662297
Epoch: 4626 cost = 0.027096050
Validation Loss: 0.04304114
Epoch: 4627 cost = 0.027071261
Validation Loss: 0.038860653
Epoch: 4628 cost = 0.027095092
Validation Loss: 0.0418987
Epoch: 4629 cost = 0.027070377
Validation Loss: 0.043158136
Epoch: 4630 cost = 0.027094180
Validation Loss: 0.052445453
Epoch: 4631 cost = 0.027069440
Validation Loss: 0.05154408
Epoch: 4632 cost = 0

Validation Loss: 0.034111086
Epoch: 4762 cost = 0.027032437
Validation Loss: 0.032841813
Epoch: 4763 cost = 0.027008369
Validation Loss: 0.035368603
Epoch: 4764 cost = 0.027031502
Validation Loss: 0.040414646
Epoch: 4765 cost = 0.027007427
Validation Loss: 0.0455365
Epoch: 4766 cost = 0.027030554
Validation Loss: 0.05819931
Epoch: 4767 cost = 0.027006486
Validation Loss: 0.037171252
Epoch: 4768 cost = 0.027029605
Validation Loss: 0.03422947
Epoch: 4769 cost = 0.027005572
Validation Loss: 0.035773877
Epoch: 4770 cost = 0.027028664
Validation Loss: 0.038077746
Epoch: 4771 cost = 0.027004641
Validation Loss: 0.039864276
Epoch: 4772 cost = 0.027027733
Validation Loss: 0.038379528
Epoch: 4773 cost = 0.027003712
Validation Loss: 0.034190428
Epoch: 4774 cost = 0.027026780
Validation Loss: 0.03256003
Epoch: 4775 cost = 0.027002771
Validation Loss: 0.03350851
Epoch: 4776 cost = 0.027025828
Validation Loss: 0.033965163
Epoch: 4777 cost = 0.027001868
Validation Loss: 0.04096246
Epoch: 4778 cost =

Epoch: 4902 cost = 0.026965944
Validation Loss: 0.058535747
Epoch: 4903 cost = 0.026943062
Validation Loss: 0.0503084
Epoch: 4904 cost = 0.026964994
Validation Loss: 0.052752748
Epoch: 4905 cost = 0.026942111
Validation Loss: 0.05699885
Epoch: 4906 cost = 0.026964036
Validation Loss: 0.051991835
Epoch: 4907 cost = 0.026941165
Validation Loss: 0.035065286
Epoch: 4908 cost = 0.026963084
Validation Loss: 0.03611215
Epoch: 4909 cost = 0.026940247
Validation Loss: 0.03281773
Epoch: 4910 cost = 0.026962123
Validation Loss: 0.03319149
Epoch: 4911 cost = 0.026939317
Validation Loss: 0.032024905
Epoch: 4912 cost = 0.026961170
Validation Loss: 0.032501057
Epoch: 4913 cost = 0.026938375
Validation Loss: 0.028945163
Epoch: 4914 cost = 0.026960224
Validation Loss: 0.035684045
Epoch: 4915 cost = 0.026937435
Validation Loss: 0.045241944
Epoch: 4916 cost = 0.026959290
Validation Loss: 0.044549577
Epoch: 4917 cost = 0.026936490
Validation Loss: 0.038008273
Epoch: 4918 cost = 0.026958308
Validation Loss

In [58]:
currentmin

0.028756049

In [54]:
togive1

[array([[0.25204715, 0.80150497, 0.02951745, 0.10669305, 0.10169823,
         0.02269761, 0.29088235, 0.14584139, 0.708738  , 0.61218274,
         0.1941818 , 0.01269354, 0.1751643 , 0.12654343]], dtype=float32),
 array([[0.26876733, 0.80278885, 0.01544675, 0.09581774, 0.09186165,
         0.01447429, 0.35852137, 0.13997771, 0.6811743 , 0.6164866 ,
         0.21612428, 0.00619126, 0.18753792, 0.1139154 ]], dtype=float32),
 array([[0.27154464, 0.80851066, 0.01878929, 0.11991043, 0.10562573,
         0.01805211, 0.3667678 , 0.12485002, 0.7016876 , 0.6158034 ,
         0.21087739, 0.00739287, 0.1939296 , 0.11161887]], dtype=float32),
 array([[0.25463548, 0.80363035, 0.02483327, 0.10641886, 0.10436142,
         0.01976654, 0.33195987, 0.13467546, 0.7008578 , 0.60653156,
         0.19142848, 0.01053145, 0.20586047, 0.12775642]], dtype=float32),
 array([[0.21282744, 0.81579167, 0.04584751, 0.10916752, 0.1094918 ,
         0.03051892, 0.2709077 , 0.12491069, 0.742136  , 0.5925941 ,
         0

In [57]:
import csv
with open('C:/Users/yy2895/Desktop/update_stresult19-14-19.csv', 'w',newline='') as myfile:
     wr = csv.writer(myfile)
     for i in range(len(togive1)):
        wr.writerow(list(togive1[i][0]))